In [3]:
# Import libraries
import numpy as np
import pandas as pd
from obspy import read
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os
import tqdm


In [2]:
!python -m pip install tqdm

     |████████████████████████████████| 78 kB 2.3 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [7]:
import pandas as pd
import tqdm

def load_data(base_data_path):

    cat_file = os.path.join(base_data_path, "lunar", "training", "catalogs", 'apollo12_catalog_GradeA_final.csv')
    cat = pd.read_csv(cat_file)
    
    l = []
    data_id = 0
    for x in tqdm.tqdm(cat.to_dict(orient='records')):
        v = {}
        v['label'] = x['time_rel(sec)']
        v['meta'] = f"evid={x['evid']};mq_type={x['mq_type']};filename={x['filename']}"
        v['ID'] = data_id
        data_id += 1

        data_path = os.path.join(base_data_path, "lunar", "training", "data", "S12_GradeA")
        data_path = os.path.join(data_path, x['filename'] + '.csv')
        if not os.path.isfile(data_path):
            print(data_path, 'not found...')
            continue

        data_cat = pd.read_csv(data_path)
        data_time_arr = data_cat["time_rel(sec)"]
        data_velocity_arr = data_cat["velocity(m/s)"]
    
        v['data_time_arr'] = data_time_arr.values
        v['data_arr'] = data_velocity_arr.values
    
        closest_index = (data_time_arr - v['label']).abs().argmin()
        label_arr = np.zeros_like(data_time_arr)
        label_arr[closest_index] = 1
        
        v['label_index'] = closest_index
        v['label_arr'] = label_arr
    
        l.append(v)

    df = pd.DataFrame(l)
    df = df[['ID', 'data_time_arr', 'data_arr', 'label', 'label_index', 'label_arr', 'meta']]

    return df


df = load_data("./space_apps_2024_seismic_detection/data/")

 26%|█████████████████████▊                                                             | 20/76 [00:12<00:33,  1.68it/s]

./space_apps_2024_seismic_detection/data/lunar/training/data/S12_GradeA/xa.s12.00.mhz.1971-04-13HR00_evid00029.csv not found...


100%|███████████████████████████████████████████████████████████████████████████████████| 76/76 [00:48<00:00,  1.58it/s]


In [8]:
df

,ID,data_time_arr,data_arr,label,label_index,label_arr,meta
0,0,"[0.0, 0.1509433962264151, 0.3018867924528302, ...","[-6.153278962788711e-14, -7.70128843364098e-14...",73500.0,486937,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",evid=evid00002;mq_type=impact_mq;filename=xa.s...
1,1,"[0.0, 0.1509433962264151, 0.3018867924528302, ...","[-5.481780117043957e-15, -6.878652555543394e-1...",12720.0,84270,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",evid=evid00003;mq_type=impact_mq;filename=xa.s...
2,2,"[0.0, 0.1509433962264151, 0.3018867924528302, ...","[-2.8212463353274303e-14, -3.5233170652581576e...",73020.0,483757,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",evid=evid00004;mq_type=impact_mq;filename=xa.s...
3,3,"[0.0, 0.1509433962264151, 0.3018867924528302, ...","[9.01642264710853e-15, 1.1305708384819468e-14,...",4440.0,29415,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",evid=evid00006;mq_type=impact_mq;filename=xa.s...
4,4,"[0.0, 0.1509433962264151, 0.3018867924528302, ...","[-1.5835653822406575e-16, -1.8729952083938926e...",52140.0,345427,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",evid=evid00007;mq_type=deep_mq;filename=xa.s12...
...,...,...,...,...,...,...,...
70,71,"[0.0, 0.1509433962264151, 0.3018867924528302, ...","[-2.4861784401436684e-16, -3.2007144155342813e...",63780.0,422542,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",evid=evid00156;mq_type=impact_mq;filename=xa.s...
71,72,"[0.0, 0.1509433962264151, 0.3018867924528302, ...","[2.3556678310214223e-14, 2.4452362696271054e-1...",65700.0,435262,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",evid=evid00191;mq_type=impact_mq;filename=xa.s...
72,73,"[0.0, 0.1509433962264151, 0.3018867924528302, ...","[8.378883424796289e-17, 7.740468608984878e-17,...",36300.0,240487,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",evid=evid00192;mq_type=impact_mq;filename=xa.s...
73,74,"[0.0, 0.1509433962264151, 0.3018867924528302, ...","[1.1799114775093931e-14, 1.4887247521375698e-1...",57780.0,382792,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",evid=evid00196;mq_type=impact_mq;filename=xa.s...


In [33]:
for x in df.to_dict(orient='records'):
    print(x)
    break


{'ID': 0, 'data_time_arr': array([0.00000000e+00, 1.50943396e-01, 3.01886792e-01, ...,
       8.64018113e+04, 8.64019623e+04, 8.64021132e+04]), 'data_arr': array([-6.15327896e-14, -7.70128843e-14, -8.39618673e-14, ...,
       -2.24030684e-14, -2.99840543e-14, -4.46551537e-14]), 'label': 73500.0, 'label_index': 486937, 'label_arr': array([0., 0., 0., ..., 0., 0., 0.]), 'meta': 'evid=evid00002;mq_type=impact_mq;filename=xa.s12.00.mhz.1970-01-19HR00_evid00002'}


In [41]:

def compute_metrics(df, df_pred):
    x1 = df[['ID', 'label']]
    x2 = df_pred[['ID', 'Pred']]

    tmp_df = x1.merge(df_pred, on='ID')
    print(tmp_df)

    tmp_df['error'] = (tmp_df['label'] - tmp_df['Pred']).abs()

    return tmp_df['error'].mean().item()



   ID    label     Pred
0   0  73500.0  73500.0
1   1  12720.0  12720.0
2   2  73020.0  73020.0


0.0